In [1]:
import pandas as pd
import numpy as np

### Expand ORF Clusters

In [2]:
gene_lists = {
    "12": ["SLC39A1", "ZBTB16"],
    "15": ["GPR176", "CHRM4", "TSC22D1", "LZTS2", "MYT1"],
    "16": ["MRPS2", "SARS2", "LDHAL6B", "ECH1", "DGUOK", "UQCRFS1"],
    "10": ["YAP1"],
}

In [3]:
orf_similarity_percentile_df = pd.read_parquet("cos_sim/cosine_similarities_percentile_genes_orf.parquet")

In [4]:
top_similar = []

for issue in gene_lists:
    genes = gene_lists[issue]
    gene_list = []
    connections_df = orf_similarity_percentile_df[genes]
    connections_df = (
        connections_df.stack().reset_index().rename(columns={0: "percentile"})
    )
    connections_df = pd.concat(
        [
            connections_df,
            connections_df.rename(columns={"gene_1": "gene_2", "gene_2": "gene_1"}),
        ],
        ignore_index=True,
    ).drop_duplicates()
    for gene in genes:
        if gene != "YAP1":
            temp = connections_df.query("gene_1 == @gene").nlargest(
                columns="percentile", n=25
            )["gene_2"].to_list() + [gene]
            gene_list.append(temp)
        else:
            temp = connections_df.query("gene_1 == @gene").nlargest(
                columns="percentile", n=25
            )["gene_2"].to_list() + [gene]
            gene_list.append(temp)

            temp = connections_df.query("gene_1 == @gene").nsmallest(
                columns="percentile", n=25
            )["gene_2"].to_list() + [gene]
            gene_list.append(temp)

    top_similar = np.unique(np.asarray(gene_list).flatten())

    if issue != "10":
        with open(f"output/top_similar_orf_{issue}.txt", "w") as f:
            f.write("\n".join(top_similar))
    else:
        with open(f"output/top_similar_antisimilar_orf_{issue}.txt", "w") as f:
            f.write("\n".join(top_similar))

### Expland CRISPR Clusters

In [5]:
gene_lists = {
    "12": ["SLC39A1", "ZBTB16"],
    "15": ["PVR", "UQCRFS1", "SARS2", "ECH1", "LAIR1", "SLC1A5"],
    "16": ["GPR176", "CHRM4", "TSC22D1", "MYT1"],
}